In [1]:
import torch
import cv2
import copy
import random
import colorsys
import requests
from io import BytesIO

import matplotlib.pyplot as plt
from  config.config import get_cfg

from data.build import (
    build_detection_test_loader,
    build_detection_train_loader,
)
from data.mapper import DatasetMapper
import data.transforms as T
from data.phase_1 import load_voc_instances,VOC_CLASS_NAMES
from data.utils import build_augmentation

In [2]:
import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image
device = 'cuda:1'

In [14]:
cfg = get_cfg()
cfg.merge_from_file('./config_files/voc.yaml')

DIR_NAME = '/data/private/OWOD/datasets/VOC2007'
split = 'train'
data = load_voc_instances(DIR_NAME,split,VOC_CLASS_NAMES,phase='t1',COCO_CLASS=False)
mapper = DatasetMapper(is_train=True, augmentations=build_augmentation(cfg,True))
data_loader = build_detection_train_loader(data,mapper=mapper,total_batch_size=1)
VOC_CLASS_NAMES_NEW = (*VOC_CLASS_NAMES, 'unknown')
loader = iter(data_loader)

In [34]:
transform = pth_transforms.Compose([
        pth_transforms.Resize(224),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
vit = torch.hub.load('facebookresearch/dino:main', 'dino_vits8').to(device)
vit.eval()
resize = pth_transforms.Resize(224)
patch_size = 8


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [84]:
batched_inputs = next(loader)
gt_box = batched_inputs[0]['instances'].gt_boxes
img = batched_inputs[0]['image']/255
img = transform(img.unsqueeze(0))
w, h = img.shape[1] - img.shape[1] % patch_size, img.shape[2] - img.shape[2] % patch_size
w_featmap = img.shape[-2] // patch_size
h_featmap = img.shape[-1] // patch_size

attentions = vit.get_last_selfattention(img.to(device))

nh = attentions.shape[1] # number of head

# we keep only the output patch attention
attentions = attentions[0, :, 0, 1:].reshape(nh, -1)


attentions = attentions.reshape(nh, w_featmap, h_featmap)
attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=patch_size, mode="nearest")[0]
attentions = resize(attentions).detach().cpu().numpy()

In [85]:
src = copy.deepcopy(batched_inputs[0]['image'])
src = resize(src).cpu().numpy()
src = src.transpose(1,2,0)
print(attentions.shape)
plt.figure(figsize=(10,10))
attn = np.mean(attentions,axis=0)
plt.imshow(src)
plt.imshow(attn,alpha=0.3,cmap='jet')
plt.axis('off')
plt.show()

(6, 224, 296)
